In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, Image, display
import tempfile
import os
import shutil

import cartopy.crs as ccrs
import shapely
import pyart
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate
from tint.visualization import embed_mp4_as_gif
import pandas as pd

from lmatools.io.LMA_h5_file import LMAh5File
from lmatools.coordinateSystems import RadarCoordinateSystem, GeographicSystem, TangentPlaneCartesianSystem, MapProjection


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



Description of fields:

Storm_id: The ID of the storm identified. A storm is defined by a continuous period where cells were identified by TINT.

Scan: The scan number for the storm

Time: time of scan

Uid: The cell id

Isolated: Is the cell isolated?

Area: Area of cell in square km

Vol: 3d volume of cell in km^3

Grid_x: x location in terms of grid indices when using the grid specification supplied in the earlier code

Grid_y: y location in terms of grid indices when using the grid specification supplied in the earlier code

Lat: Cell center latitude

Lon: Cell center longitude

Max: Reflectivity maximum

Max_alt: The maximum altitude of the cell

Kdp_pct: Mean KDP in KDP column

Zdr_pct: Mean ZDR in ZDR column

Zhh_pct: Mean reflectivity in KDP column

`csu_dsd` - DSD parameter estimation via several different methodologies

`csu_kdp` - An FIR-based KDP estimation algorithm

Kdp_pet: KDP potential energy

Zdr_pet: ZDR potential energy

Zhh_pet: Reflectivity potential energy


In [2]:
import glob
filenames = sorted(glob.glob('/home/jessica/tracer/Tgri*.nc'))
filenames
# radar = pyart.io.read_cfradial(arq[0])

['/home/jessica/tracer/Tgrid_000.nc',
 '/home/jessica/tracer/Tgrid_001.nc',
 '/home/jessica/tracer/Tgrid_002.nc',
 '/home/jessica/tracer/Tgrid_003.nc',
 '/home/jessica/tracer/Tgrid_004.nc',
 '/home/jessica/tracer/Tgrid_005.nc',
 '/home/jessica/tracer/Tgrid_006.nc',
 '/home/jessica/tracer/Tgrid_007.nc',
 '/home/jessica/tracer/Tgrid_008.nc',
 '/home/jessica/tracer/Tgrid_009.nc',
 '/home/jessica/tracer/Tgrid_010.nc',
 '/home/jessica/tracer/Tgrid_011.nc',
 '/home/jessica/tracer/Tgrid_012.nc',
 '/home/jessica/tracer/Tgrid_013.nc',
 '/home/jessica/tracer/Tgrid_014.nc',
 '/home/jessica/tracer/Tgrid_015.nc',
 '/home/jessica/tracer/Tgrid_016.nc',
 '/home/jessica/tracer/Tgrid_017.nc',
 '/home/jessica/tracer/Tgrid_018.nc',
 '/home/jessica/tracer/Tgrid_019.nc',
 '/home/jessica/tracer/Tgrid_020.nc',
 '/home/jessica/tracer/Tgrid_021.nc']

In [3]:
# Now we can easily instantiate generators of these grids like so
grids = (pyart.io.read_grid(fn) for fn in filenames)

# First, let's instantiate a tracks object and view the default parameters
tracks_obj = Cell_tracks()
tracks_obj.params

# The cells we're interested in look a bit small. Let's reduce the minimum size threshold.
tracks_obj.params['MIN_SIZE'] = 4
# We'll give the generator of grids we made earlier to the get_tracks method of our tracks object.
tracks_obj.get_tracks(grids)

Writing tracks for scan 0
Writing tracks for scan 1
Writing tracks for scan 2
Writing tracks for scan 3
Writing tracks for scan 4
Writing tracks for scan 5
Writing tracks for scan 6
Writing tracks for scan 7
Writing tracks for scan 8
Writing tracks for scan 9
Writing tracks for scan 10
Writing tracks for scan 11
Writing tracks for scan 12
Writing tracks for scan 13
Writing tracks for scan 14
Writing tracks for scan 15
Writing tracks for scan 16
Writing tracks for scan 17
Writing tracks for scan 18
Writing tracks for scan 19
Writing tracks for scan 20
Writing tracks for scan 21


time elapsed 0.4 minutes


In [4]:
# Now we can view the 'tracks' attribute of our tracks object to see the results.
tracks_obj.tracks

time   grid_x   grid_y      lon      lat   area  \
scan uid                                                                      
0    0   2017-07-08 20:31:36.294  159.625   10.250 -95.4852  27.7626   24.0   
     1   2017-07-08 20:31:36.294  302.643   32.929 -94.0301  27.9659   14.0   
     2   2017-07-08 20:31:36.294   36.643  146.119 -96.7544  28.9758   42.0   
     3   2017-07-08 20:31:36.294   46.500  143.875 -96.6616  28.9589    8.0   
     4   2017-07-08 20:31:36.294   58.500  145.000 -96.5384  28.9693    4.0   
...                          ...      ...      ...      ...      ...    ...   
21   127 2017-07-08 22:26:07.730  167.519  379.346 -95.4147  31.0813   52.0   
     137 2017-07-08 22:26:07.730  268.500  380.500 -94.3647  31.0888    4.0   
     130 2017-07-08 22:26:07.730  257.800  385.083 -94.4694  31.1343   60.0   
     100 2017-07-08 22:26:07.730  161.976  393.119 -95.4782  31.2070  168.0   
     103 2017-07-08 22:26:07.730  176.546  388.420 -95.3204  31.1624  119.0   

            vol        max  max_alt  isolated  
scan uid                                       
0    0     75.5  45.192131      9.0      True  
     1     35.5  43.095295      5.0      True  
     2    191.0  54.453011      7.5     False  
     3     12.5  35.869747      4.0     False  
     4      4.0  35.000370      3.5     False  
...         ...        ...      ...       ...  
21   127  232.0  51.538857      8.0     False  
     137    2.5  34.088097      2.5     False  
     130  314.0  49.966209     12.0     False  
     100  813.0  54.834606     13.0     False  
     103  585.0  51.472496     12.0     False  

[483 rows x 10 columns]

In [22]:
# Let's find the cells that were tracked for the most frames
tracks_obj.tracks.groupby(level='uid').size().sort_values(ascending=False)[:16]

uid
18     22
21     22
41     19
20     18
50     18
71     13
10     13
30     12
100     9
8       9
84      8
103     8
91      8
24      7
80      7
54      7
dtype: int64

In [23]:
# We can view the attributes of this cell throughout its lifetime
tracks_obj.tracks.xs('54', level='uid')

,time,grid_x,grid_y,lon,lat,area,vol,max,max_alt,isolated
scan,,,,,,,,,,
5,2017-07-08 21:00:33.261,131.375,364.750,-95.8022,30.9538,8.0,20.5,40.500000,4.5,True
6,2017-07-08 21:06:20.576,130.952,365.619,-95.8023,30.9628,21.0,71.5,44.919159,8.0,False
7,2017-07-08 21:12:07.965,132.000,364.133,-95.7917,30.9449,15.0,36.0,40.731735,8.0,False
8,2017-07-08 21:17:28.855,132.375,363.312,-95.7916,30.9359,16.0,47.5,48.012028,8.0,False
9,2017-07-08 21:22:50.271,133.455,363.455,-95.7811,30.9360,11.0,24.5,41.011459,4.5,False
10,2017-07-08 21:27:44.359,134.143,364.429,-95.7707,30.9450,14.0,31.5,41.136204,4.5,False
11,2017-07-08 21:32:38.697,134.333,365.500,-95.7709,30.9630,6.0,9.0,34.248074,4.5,False


In [ ]:
fig = plt.figure(figsize=(20, 15))

        fig.suptitle('Cell ' + uid + ' Scan ' + str(nframe), fontsize=22)
        plt.axis('off')

        # Lagrangian View
        ax = fig.add_subplot(3, 2, (1, 3), projection=projection)

        display.plot_grid(field, level=get_grid_alt(grid_size, alt),
                          vmin=vmin, vmax=vmax, mask_outside=False,
                          cmap=cmap, colorbar_flag=False,
                          ax=ax, projection=projection)

        display.plot_crosshairs(lon=lon, lat=lat, linestyle='--', 
                                color='k', linewidth=3, ax=ax)

        ax.set_xlim(lvxlim[0], lvxlim[1])
        ax.set_ylim(lvylim[0], lvylim[1])

        ax.set_xticks(np.arange(lvxlim[0], lvxlim[1], stepsize))
        ax.set_yticks(np.arange(lvylim[0], lvylim[1], stepsize))

        ax.set_title('Top-Down View', fontsize=title_font)
        ax.set_xlabel('Longitude of grid cell center\n [degree_E]',
                       fontsize=axes_font)
        ax.set_ylabel('Latitude of grid cell center\n [degree_N]',
                       fontsize=axes_font)

        # Latitude Cross Section
        ax = fig.add_subplot(3, 2, 2)
        display.plot_latitude_slice(field, lon=lon, lat=lat,
                                    title_flag=False,
                                    colorbar_flag=False, edges=False,
                                    vmin=vmin, vmax=vmax, mask_outside=False,
                                    cmap=cmap,
                                    ax=ax)

        ax.set_xlim(xlim[0], xlim[1])
        ax.set_xticks(np.arange(xlim[0], xlim[1], 6))
        ax.set_xticklabels(np.round((np.arange(xlim[0], xlim[1], 6)),
                                     2))

        ax.set_title('Latitude Cross Section', fontsize=title_font)
        ax.set_xlabel('East West Distance From Origin (km)' + '\n',
                       fontsize=axes_font)
        ax.set_ylabel('Distance Above Origin (km)', fontsize=axes_font)
        ax.set_aspect(aspect=1.3)

        # Longitude Cross Section
        ax = fig.add_subplot(3, 2, 4)
        display.plot_longitude_slice(field, lon=lon, lat=lat,
                                     title_flag=False,
                                     colorbar_flag=False, edges=False,
                                     vmin=vmin, vmax=vmax, mask_outside=False,
                                     cmap=cmap,
                                     ax=ax)
        ax.set_xlim(ylim[0], ylim[1])
        ax.set_xticks(np.arange(ylim[0], ylim[1], 6))
        ax.set_xticklabels(np.round(np.arange(ylim[0], ylim[1], 6), 2))

        ax.set_title('Longitudinal Cross Section', fontsize=title_font)
        ax.set_xlabel('North South Distance From Origin (km)',
                       fontsize=axes_font)
        ax.set_ylabel('Distance Above Origin (km)', fontsize=axes_font)
        ax.set_aspect(aspect=1.3)

        # Time Series Statistic
        max_field = cell['max']
        plttime = cell['time']

        # Plot
        ax = fig.add_subplot(3, 2, (5, 6))
        ax.plot(plttime, max_field, color='b', linewidth=3)
        ax.axvline(x=plttime[nframe], linewidth=4, color='r')
        ax.set_title('Time Series', fontsize=title_font)
        ax.set_xlabel('Time (UTC) \n Lagrangian Viewer Time',
                       fontsize=axes_font)
        ax.set_ylabel('Maximum ' + field, fontsize=axes_font)

        # plot and save figure
        fig.savefig(tmp_dir + '/frame_' + str(nframe).zfill(3) + '.png')
        plt.close()
        del grid, display
        gc.collect()

In [ ]:
arq = glob.glob('/home/jessica/Documentos/OK/OK_radar/usados/KTLX20171022_021931_V06_proc.nc')
radar1 = pyart.io.read_cfradial(arq[0])
fname = arq[0]

print(radar1.fields.keys())
# print(radar2.fields.keys())
    
d1 = date(int(fname[48:52]),int(fname[52:54]),int(fname[54:56]))
t1 = time(int(fname[57:59]),int(fname[59:61]),int(fname[61:63]))
tt1 = datetime.combine(d1,t1)
tt1.strftime("%d/%m/%y %H:%M:%S")
# tt2 = datetime.combine(d2,t2)
# tt2.strftime("%d/%m/%y %H:%M:%S")
elevations = radar1.fixed_angle['data']
radar1.fixed_angle['data']
    
hidcolor = dualpol.HidColors()

radar_list1 = [radar1]
grid1 = pyart.map.grid_from_radars(radar_list1,
    grid_shape=(31, 501, 501),  
    grid_limits=((0, 15000.0), (-250000, 250000), (-250000, 250000)),
    grid_origin = (radar1.latitude['data'][0], radar1.longitude['data'][0]),
                                  fields=['reflectivity', 'differential_reflectivity', 
                                          'KDP_CSU','cross_correlation_ratio', 'temperature'],
                                  gridding_algo='map_gates_to_grid', grid_origin_alt=0)
